# 2-S1: MLflow Tracking 기초

MLflow를 사용한 **실험 추적**을 학습합니다.

## 학습 목표
1. 왜 실험 추적이 필요한가? (Phase 1 문제점 연결)
2. MLflow Tracking: Experiment, Run, Artifact
3. Autolog로 자동 기록
4. Autolog + 수동 로깅 조합 (실무 패턴)

## 예상 시간
- 약 1시간

## 선수 조건
- Phase 1 완료 (XGBoost 학습 경험)
- `pip install mlflow xgboost scikit-learn`

## 다음 학습
- 2-S2: Model Registry (버전 관리, Alias, Champion/Challenger)

In [ ]:
# 패키지 설치 (필요시)
# !pip install mlflow xgboost scikit-learn

import mlflow
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score
from sklearn.datasets import make_classification

print(f"MLflow 버전: {mlflow.__version__}")
print(f"XGBoost 버전: {xgb.__version__}")

# 전역 autolog 비활성화 (학습 목적으로 수동 로깅부터 배움)
mlflow.autolog(disable=True)
print("\n전역 autolog 비활성화됨 (학습용)")

In [ ]:
# 실습용 데이터 생성 (불균형 이진 분류)
X, y = make_classification(
    n_samples=10000,
    n_features=20,
    n_informative=10,
    n_redundant=5,
    weights=[0.95, 0.05],  # 5% 양성 클래스 (사기)
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"학습 데이터: {len(X_train)}건")
print(f"테스트 데이터: {len(X_test)}건")
print(f"양성 비율: {y_train.mean():.2%}")

---
## 1. 왜 실험 추적이 필요한가?

### 1.1 Phase 1에서 겪은 문제

Phase 1에서 XGBoost를 학습할 때 이런 경험 있으셨나요?

```
[문제 상황]

월요일: learning_rate=0.1, max_depth=6 -> AUC 0.85
화요일: learning_rate=0.05, max_depth=8 -> AUC 0.87
수요일: learning_rate=???, max_depth=??? -> AUC 0.89

금요일: "수요일 모델이 제일 좋았는데, 하이퍼파라미터 뭐였지?"
```

**수동 기록의 한계:**
- 엑셀/노트에 기록 -> 실수, 누락
- 파일명으로 구분 -> `model_v1.pkl`, `model_final.pkl`, `model_final_v2.pkl`
- 재현 불가능 -> "그때 왜 그 결과가 나왔지?"

In [ ]:
# 수동 기록의 문제점 시뮬레이션

# 이렇게 수동으로 기록하면...
experiment_log = []

# 실험 1
experiment_log.append({
    'date': '월요일',
    'lr': 0.1,
    'depth': 6,
    'auc': 0.85
})

# 실험 2
experiment_log.append({
    'date': '화요일',
    'lr': 0.05,
    'depth': 8,
    'auc': 0.87
})

# 실험 3 - 실수로 기록 안 함!
# experiment_log.append({...})  <- 깜빡함

print("수동 기록 결과:")
for exp in experiment_log:
    print(f"  {exp}")

print("\n문제점:")
print("  - 실험 3 기록 누락")
print("  - 모델 파일 위치 없음")
print("  - 재현 불가능")

### 1.2 MLflow가 해결하는 것

MLflow는 ML 실험의 **모든 것을 자동으로 기록**합니다:

| 항목 | 수동 기록 | MLflow |
|------|----------|--------|
| 파라미터 | 엑셀/노트 | 자동 저장 |
| 메트릭 | 기억에 의존 | 자동 저장 + 시각화 |
| 모델 파일 | `model_final_v3.pkl` | 버전별 관리 |
| 재현성 | 불가능 | 코드 + 환경 저장 |
| 비교 | 직접 비교 | UI에서 한눈에 |

### 1.3 MLflow 4가지 컴포넌트

```
+-------------------------------------------------------+
|                      MLflow                            |
+-------------+-------------+-------------+--------------+
|  Tracking   |  Registry   |  Projects   |   Serving    |
|  실험 기록   |  모델 저장소 |  재현 환경   |   모델 배포   |
|  <- 오늘!   |  <- 2-S2!   |  (선택)     |   (선택)     |
+-------------+-------------+-------------+--------------+
```

**오늘 배울 것: Tracking** (실험 파라미터, 메트릭 기록)

---
## 2. MLflow Tracking 기초

### 2.1 핵심 개념

```
Experiment (실험 그룹) - "FDS-XGBoost-튜닝"
|
+-- Run 1 (한 번의 학습)
|   +-- Parameters: {learning_rate: 0.1, max_depth: 6}
|   +-- Metrics: {auc: 0.85, recall: 0.80}
|   +-- Artifacts: model.pkl, feature_importance.png
|
+-- Run 2
|   +-- Parameters: {learning_rate: 0.05, max_depth: 8}
|   +-- Metrics: {auc: 0.87, recall: 0.82}
|   +-- Artifacts: model.pkl
|
+-- Run 3
    +-- ...
```

| 용어 | 설명 | 예시 |
|------|------|------|
| **Experiment** | 실험 그룹 | "FDS-XGBoost-튜닝" |
| **Run** | 한 번의 학습 | Optuna 50회 = 50 Runs |
| **Parameter** | 입력 설정 | learning_rate, max_depth |
| **Metric** | 결과 수치 | AUC, Recall, AUPRC |
| **Artifact** | 결과 파일 | model.pkl, plots |

In [ ]:
# MLflow Tracking 기본 예제

import mlflow

# 1. 실험 설정 (없으면 자동 생성)
mlflow.set_experiment("FDS-Study-Tracking")

# 2. Run 시작
with mlflow.start_run(run_name="tracking-basic"):
    
    # 3. 파라미터 기록
    mlflow.log_param("learning_rate", 0.1)
    mlflow.log_param("max_depth", 6)
    mlflow.log_param("n_estimators", 100)
    
    # 4. 메트릭 기록
    mlflow.log_metric("auc", 0.85)
    mlflow.log_metric("recall", 0.80)
    mlflow.log_metric("precision", 0.75)
    
    # Run ID 확인
    run_id = mlflow.active_run().info.run_id
    print(f"Run ID: {run_id}")

print("\n기록 완료!")
print("MLflow UI에서 확인: mlflow ui 명령 실행 후 http://localhost:5000")

### 2.2 실제 모델 학습 + 기록

In [ ]:
# 실제 XGBoost 학습 + MLflow 기록

mlflow.set_experiment("FDS-Study-Tracking")

# 하이퍼파라미터 설정
params = {
    'learning_rate': 0.1,
    'max_depth': 6,
    'n_estimators': 50,
    'scale_pos_weight': (y_train == 0).sum() / (y_train == 1).sum(),
    'random_state': 42,
    'eval_metric': 'logloss'
}

with mlflow.start_run(run_name="xgboost-manual-logging"):
    
    # 파라미터 기록
    for key, value in params.items():
        mlflow.log_param(key, value)
    
    # 모델 학습
    model = xgb.XGBClassifier(**params)
    model.fit(X_train, y_train)
    
    # 예측 및 평가
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]
    
    # 메트릭 계산
    auc = roc_auc_score(y_test, y_proba)
    recall = recall_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    
    # 메트릭 기록
    mlflow.log_metric("auc", auc)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("accuracy", accuracy)
    
    # FDS 비용 지표 (커스텀)
    # FN 비용 100만원, FP 비용 5만원
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y_test, y_pred)
    fn_cost = cm[1, 0] * 1000000  # 사기 놓침
    fp_cost = cm[0, 1] * 50000    # 오탐
    total_cost = fn_cost + fp_cost
    mlflow.log_metric("total_cost", total_cost)
    
    print(f"AUC: {auc:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"총 비용: {total_cost:,}원")
    print(f"\nRun ID: {mlflow.active_run().info.run_id}")

### 실습 1: 수동 로깅

다른 하이퍼파라미터로 학습하고 MLflow에 기록하세요.

In [ ]:
# 실습 1: 수동 로깅

mlflow.set_experiment("FDS-Study-Tracking")

# TODO: 다른 하이퍼파라미터 설정
params_v2 = {
    'learning_rate': 0.05,      # TODO: 변경해보세요
    'max_depth': 8,             # TODO: 변경해보세요
    'n_estimators': 100,        # TODO: 변경해보세요
    'scale_pos_weight': (y_train == 0).sum() / (y_train == 1).sum(),
    'random_state': 42,
    'eval_metric': 'logloss'
}

with mlflow.start_run(run_name="xgboost-practice-1"):
    
    # TODO: 파라미터 기록 (힌트: mlflow.log_param())
    for key, value in params_v2.items():
        mlflow.log_param(key, value)
    
    # TODO: 모델 학습
    model_v2 = xgb.XGBClassifier(**params_v2)
    model_v2.fit(X_train, y_train)
    
    # TODO: 예측 및 평가
    y_pred_v2 = model_v2.predict(X_test)
    y_proba_v2 = model_v2.predict_proba(X_test)[:, 1]
    
    # TODO: 메트릭 계산 및 기록
    auc_v2 = roc_auc_score(y_test, y_proba_v2)
    recall_v2 = recall_score(y_test, y_pred_v2)
    
    mlflow.log_metric("auc", auc_v2)
    mlflow.log_metric("recall", recall_v2)
    
    practice1_run_id = mlflow.active_run().info.run_id
    print(f"AUC: {auc_v2:.4f}")
    print(f"Recall: {recall_v2:.4f}")
    print(f"Run ID: {practice1_run_id}")

In [ ]:
# 체크포인트 1
assert practice1_run_id is not None, "Run ID가 없습니다. MLflow Run을 실행하세요."
assert auc_v2 > 0.5, "AUC가 너무 낮습니다. 모델 학습을 확인하세요."

print("체크포인트 1 통과!")

---
## 3. Autolog - 자동 기록

### 3.1 수동 로깅 vs Autolog

```python
# 수동 로깅 - 일일이 기록
mlflow.log_param("learning_rate", 0.1)
mlflow.log_param("max_depth", 6)
mlflow.log_param("n_estimators", 100)
# ... 10줄 더...

# Autolog - 한 줄로 끝!
mlflow.sklearn.autolog()  # XGBClassifier용
```

**중요: Autolog 종류**

| 사용하는 API | Autolog 함수 |
|-------------|--------------|
| `XGBClassifier`, `XGBRegressor` (sklearn API) | `mlflow.sklearn.autolog()` |
| `xgb.train()` (native API) | `mlflow.xgboost.autolog()` |

```python
# 잘못된 사용 (에러 발생!)
mlflow.xgboost.autolog()
model = XGBClassifier()  # sklearn API인데 xgboost autolog 사용

# 올바른 사용
mlflow.sklearn.autolog()
model = XGBClassifier()  # sklearn API -> sklearn autolog
```

**Autolog가 자동으로 기록하는 것:**
- 모든 하이퍼파라미터
- 학습 메트릭 (loss 등)
- 모델 파일
- Feature Importance

In [ ]:
# Autolog 예제

mlflow.set_experiment("FDS-Study-Autolog")

# sklearn autolog 활성화 (XGBClassifier는 sklearn API 사용)
mlflow.sklearn.autolog()

with mlflow.start_run(run_name="xgboost-autolog"):
    
    # 모델 학습만 하면 됨!
    model_auto = xgb.XGBClassifier(
        n_estimators=50,
        learning_rate=0.1,
        max_depth=6,
        scale_pos_weight=(y_train == 0).sum() / (y_train == 1).sum(),
        random_state=42,
        eval_metric='logloss'
    )
    model_auto.fit(X_train, y_train)
    
    # 자동으로 기록됨:
    # - 모든 하이퍼파라미터
    # - 모델 파일
    # - Feature importance
    
    autolog_run_id = mlflow.active_run().info.run_id
    print(f"\nAutolog 완료!")
    print(f"Run ID: {autolog_run_id}")
    print("\n자동으로 기록된 것:")
    print("  - 모든 하이퍼파라미터")
    print("  - 모델 파일 (model/)")
    print("  - Feature importance")

### 3.2 Autolog의 한계

**Autolog가 기록하지 않는 것:**
- 커스텀 메트릭 (FDS 비용, AUPRC 등)
- 테스트 데이터 평가 결과
- 비즈니스 지표

**-> 실무에서는 Autolog + 수동 로깅 조합 사용!**

**Autolog 주의사항:**
- `XGBClassifier` 사용 시 -> `mlflow.sklearn.autolog()`
- `xgb.train()` 사용 시 -> `mlflow.xgboost.autolog()`
- 잘못 사용하면 `_estimator_type undefined` 에러 발생!

In [ ]:
# Autolog + 수동 로깅 조합 (실무 패턴)

mlflow.set_experiment("FDS-Study-Autolog")

# sklearn autolog (XGBClassifier용)
mlflow.sklearn.autolog()

with mlflow.start_run(run_name="xgboost-autolog-custom"):
    
    # 모델 학습 (Autolog가 자동 기록)
    model_combo = xgb.XGBClassifier(
        n_estimators=50,
        learning_rate=0.1,
        max_depth=6,
        scale_pos_weight=(y_train == 0).sum() / (y_train == 1).sum(),
        random_state=42,
        eval_metric='logloss'
    )
    model_combo.fit(X_train, y_train)
    
    # 수동으로 커스텀 메트릭 추가
    y_pred_combo = model_combo.predict(X_test)
    y_proba_combo = model_combo.predict_proba(X_test)[:, 1]
    
    # 테스트 메트릭
    test_auc = roc_auc_score(y_test, y_proba_combo)
    test_recall = recall_score(y_test, y_pred_combo)
    mlflow.log_metric("test_auc", test_auc)
    mlflow.log_metric("test_recall", test_recall)
    
    # FDS 비용 지표 (커스텀)
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y_test, y_pred_combo)
    total_cost = cm[1, 0] * 1000000 + cm[0, 1] * 50000
    mlflow.log_metric("total_cost", total_cost)
    
    combo_run_id = mlflow.active_run().info.run_id
    print(f"Test AUC: {test_auc:.4f}")
    print(f"Test Recall: {test_recall:.4f}")
    print(f"총 비용: {total_cost:,}원")
    print(f"\nRun ID: {combo_run_id}")

### 실습 2: Autolog 사용

Autolog를 사용하여 학습하고, 커스텀 메트릭을 추가하세요.

In [ ]:
# 실습 2: Autolog + 커스텀 메트릭

mlflow.set_experiment("FDS-Study-Autolog")

# sklearn autolog (XGBClassifier용)
mlflow.sklearn.autolog()

with mlflow.start_run(run_name="practice-autolog"):
    
    # TODO: 다른 하이퍼파라미터로 모델 학습
    model_practice = xgb.XGBClassifier(
        n_estimators=100,       # TODO: 변경해보세요
        learning_rate=0.05,     # TODO: 변경해보세요
        max_depth=8,            # TODO: 변경해보세요
        scale_pos_weight=(y_train == 0).sum() / (y_train == 1).sum(),
        random_state=42,
        eval_metric='logloss'
    )
    model_practice.fit(X_train, y_train)
    
    # TODO: 테스트 데이터 예측
    y_pred_practice = model_practice.predict(X_test)
    y_proba_practice = model_practice.predict_proba(X_test)[:, 1]
    
    # TODO: 테스트 메트릭 계산 및 기록
    practice_auc = roc_auc_score(y_test, y_proba_practice)
    practice_recall = recall_score(y_test, y_pred_practice)
    
    mlflow.log_metric("test_auc", practice_auc)
    mlflow.log_metric("test_recall", practice_recall)
    
    practice2_run_id = mlflow.active_run().info.run_id
    print(f"Test AUC: {practice_auc:.4f}")
    print(f"Test Recall: {practice_recall:.4f}")

In [ ]:
# 체크포인트 2
assert practice2_run_id is not None, "Run ID가 없습니다."
assert practice_auc > 0.5, "AUC가 너무 낮습니다."
assert practice_recall > 0, "Recall을 계산하세요."

print("체크포인트 2 통과!")

---
## 4. 최종 요약

In [ ]:
print("="*60)
print("  2-S1 완료: MLflow Tracking 기초")
print("="*60)
print()
print("배운 것:")
print()
print("1. 왜 MLflow가 필요한가?")
print("   - 실험 재현성")
print("   - 하이퍼파라미터/메트릭 자동 기록")
print()
print("2. MLflow Tracking")
print("   - Experiment -> Run -> Artifact 구조")
print("   - log_param(), log_metric()")
print()
print("3. Autolog")
print("   - mlflow.sklearn.autolog() (XGBClassifier용)")
print("   - 자동 기록 + 커스텀 메트릭 조합")
print()
print("="*60)
print("다음: 2-S2 Model Registry로!")
print("="*60)

### 학습 체크리스트

| 항목 | 이해도 |
|------|--------|
| Experiment, Run, Artifact 구조 | |
| log_param(), log_metric() 사용법 | |
| Autolog 사용법 및 한계 | |
| Autolog + 수동 로깅 조합 (실무) | |

### 면접 예상 질문

**1. "MLflow Tracking은 무엇을 기록하나요?"**

답변:
- **파라미터**: 모델 학습에 사용한 하이퍼파라미터 (learning_rate, max_depth 등)
- **메트릭**: 모델 성능 지표 (AUC, Recall, 비용 등)
- **아티팩트**: 모델 파일, 그래프, 데이터 샘플 등

---

**2. "Autolog의 장단점은?"**

답변:
- **장점**: 편리함, 일관성, 모든 하이퍼파라미터 자동 기록
- **단점**: 커스텀 메트릭(FDS 비용, AUPRC) 수동 추가 필요
- **실무**: Autolog + 수동 로깅 조합 사용
- **주의**: XGBClassifier는 `mlflow.sklearn.autolog()`, xgb.train()은 `mlflow.xgboost.autolog()` 사용